In [2]:
!pip install rank_bm25 -q

In [15]:
!pip install transformers -q

In [18]:
!pip install torch==1.4.0

ERROR: Could not find a version that satisfies the requirement torch==1.4.0 (from versions: 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0, 1.10.1, 1.10.2, 1.11.0)
ERROR: No matching distribution found for torch==1.4.0


In [1]:
import collections
import glob
import json
from rank_bm25 import BM25Okapi
import random
import stanza
import tqdm
import numpy as np
random.seed(131)

/work/pyelugam_umass_edu/miniconda3/envs/bm25/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
from rank_bm25 import BM25L,BM25Plus, 

In [4]:
!pip install stanza -q

In [17]:
with open("./data/nltk_stopwords.json", "r") as f:
  STOPWORDS = json.load(f)

In [18]:
STANZA_PIPELINE = stanza.Pipeline("en",
                                  processors="tokenize,lemma",
                                  tokenize_no_ssplit=True)

2022-05-12 09:42:00 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| lemma     | combined |

2022-05-12 09:42:01 INFO: Use device: gpu
2022-05-12 09:42:01 INFO: Loading: tokenize
2022-05-12 09:42:05 INFO: Loading: lemma
2022-05-12 09:42:06 INFO: Done loading processors!


In [6]:
for filename in glob.glob('./data/wikidata/*'):
    print(filename)
    file = open(filename, "r")
    doclist = [ line for line in file ]

./data/wikidata/wikisent2.txt


In [14]:
def stanza_preprocess(sentences):
    batch_len = 1000
    f = open('lemmati.txt', 'w')
    k = int(len(sentences) / batch_len)
    lemmatized = []
    for i in tqdm.tqdm(range(0, k)):
        l = i * batch_len
        batched_sent = doclist[l:l+batch_len]
        doc = STANZA_PIPELINE("\n\n".join(batched_sent))
        for sentence in doc.sentences:
            sentence_lemmas = []
            for token in sentence.tokens:
                (token_dict,) = token.to_dict()
                if "lemma" in token_dict:
                    maybe_lemma = token_dict["lemma"].lower()
                    if maybe_lemma not in STOPWORDS:
                        sentence_lemmas.append(maybe_lemma)
                        
            lemmatized.append(sentence_lemmas)
            f.write(str(sentence_lemmas))
            f.write('\n')
    
    f.close()
    return lemmatized

In [36]:
me = doclist[:200000]

In [37]:
lemmatized = stanza_preprocess([x for x in (me)])

100%|██████████| 200/200 [17:56<00:00,  5.38s/it]


In [48]:
model = BM25Okapi(lemmatized)

In [77]:
import pickle

#To save bm25 object
with open('bm25result', 'wb') as bm25result_file:
    pickle.dump(model, bm25result_file)

In [78]:
#to read bm25 object
with open('bm25result', 'rb') as bm25result_file:
    bm25result = pickle.load(bm25result_file)

In [79]:

query = "film actor"
tokenized_query = query.split(" ")
bm25result.get_top_n(tokenized_query, me, n=1)


['Actors Gus Weinburg and Alice Gale are the only actors in the film that appeared in the play.\n']

In [54]:
from collections import defaultdict
from tqdm import tqdm

In [73]:
def get_finetuning_data_wikipedia(bm25model, triple_path, statement_path, finetune_data_path, wiki_sentences_path, debug=False, num_processes=48):
    with open(triple_path, 'r', encoding='utf-8') as fin_triple, open(statement_path, 'r', encoding='utf-8') as fin_state, open(finetune_data_path, 'w') as fout:
        lines_triples = fin_triple.readlines()
        lines_state = fin_state.readlines()
        questions = {}
        for j, line in enumerate(lines_state):
            dic = json.loads(line)
            questions[dic['metadata']['id']] = dic['question']['normalized']
        if debug:
            lines_triples = lines_triples[0:1]
        gpt_input_strings = []
        corpus = me
        question_answer_related_wiki_sentences = defaultdict(lambda: defaultdict(list))
        answer_set = set()
        for  line in tqdm(lines_triples):
            dic = json.loads(line)
            for qid in dic:
                answers_and_triples = dic[qid]
                question = questions[qid]
                for answer in answers_and_triples:
                    triples = answers_and_triples[answer]
                    # Take the top 10 triples
                    triples = triples[:10]
                    for triple in triples:
                        
                        triple_query = triple.split('__')
                        for ans in answer.split(' '):
                            triple_query.append(ans)
                        result_sent = bm25model.get_top_n(triple_query, corpus, n=5)
                        for ans in result_sent:
                            if ans not in answer_set:
                                print(ans)
#                                 fout.write(ans)
#                                 fout.write('\n')
                                answer_set.add(ans)
                    
                    

In [74]:
out_file ="../data/protoqa/wikipedia/finetune_train_10K_questions_1_bm25_{}.txt"

In [75]:
get_finetuning_data_wikipedia(model, "../data/protoqa/triples/wikipedia.train.10.jsonl",
      "../data/protoqa/statement/train.statement.jsonl", out_file , "../data/protoqa/wikipedia/wiki_train_sentences.txt", debug=True, num_processes=48)



  0%|          | 0/1 [00:00<?, ?it/s]

A fire umbrella is a fire dancing tool.

A cocktail umbrella is a small umbrella or parasol made from paper, paperboard, and a toothpick, used as a garnish or decoration in cocktails, desserts or other food and beverages.

Africanis is also an umbrella name for all the aboriginal dogs in Southern Africa.

A confederation, a league of sovereign states, which might or might not include nation-states.

Additionally, some umbrella schools follow a specific faith, while others are secular.

Adults have a broad pattern of shades of brown on their wings.

78 in the Shade is the sixth studio album by Small Faces.

After leaving the club he played with Sunshine.

A derivative form of the California Sound was later classified as sunshine pop.

After heavy rain the water can be colored.

Adults have been observed on the fallen fruit of umbrella trees (Musanga cecropioides).

2017 Atlantic Sun Women's Basketball Tournament was the 31st edition of the Atlantic Sun Conference Championship.

Africa i

100%|██████████| 1/1 [00:10<00:00, 10.60s/it]


In [62]:

from multiprocessing import Pool
!pip install datasets -q

from datasets import load_dataset, Features, Value

In [68]:

def contains_word(s, o, example):
    ex = example['text'].lower()
    ex_words = set(ex.split(' '))
    if s in ex_words and o in ex_words:
        return True
    else:
        return False
    
def get_wiki_sentences_for_triple(triple):
    sub, obj = triple.split('___')
    if sub in stopwords or obj in stopwords:
        return []
    sub_filter = dataset.filter(lambda example: contains_word(sub.lower(),obj.lower(),  example) , num_proc =200)
    ans = sub_filter['text'][0] if len(sub_filter) else []
    return ans

def get_finetuning_data_wikipedia(triple_path, statement_path, finetune_data_path, wiki_sentences_path, debug=False, num_processes=48):

    with open(triple_path, 'r', encoding='utf-8') as fin_triple, open(statement_path, 'r', encoding='utf-8') as fin_state:
        lines_triples = fin_triple.readlines()
        lines_state = fin_state.readlines()
        questions = {}
        for j, line in enumerate(lines_state):
            dic = json.loads(line)
            questions[dic['metadata']['id']] = dic['question']['normalized']
        if debug:
            lines_triples = lines_triples[0:1]
        gpt_input_strings = []
        question_answer_related_wiki_sentences = defaultdict(lambda: defaultdict(list))
        for j, line in enumerate(lines_triples):
            dic = json.loads(line)
            for qid in dic:
                answers_and_triples = dic[qid]
                question = questions[qid]
                for answer in answers_and_triples:
                    triples = answers_and_triples[answer]
                    # Take the top 10 triples
                    triples = triples[:10]
                    sent_str = ''
                    res1 = [get_wiki_sentences_for_triple(triple) for triple in (triples)]
                    sents = list(itertools.chain(*res1))
                    for sent in sents:
                        question_answer_related_wiki_sentences[qid][answer].append(sent)
                        sent_str += sent +  ' '
                    input_string = sent_str + ' ' + question + '\t' + answer
                    print(input_string)
                    gpt_input_strings.append(input_string)
                    
#         # Write the input strings to a file
#         with open(finetune_data_path, 'w') as fout:
#             for sentence in gpt_input_strings:
#                 fout.write(sentence)
#                 fout.write('\n')

#         with open(wiki_sentences_path, 'w') as fout:
#             for key in question_answer_related_wiki_sentences:
#                 json.dump({key: question_answer_related_wiki_sentences[key]}, fout)
#                 fout.write('\n')
                
                    

with open("gist_stopwords.txt", "r") as fin:
    content = fin.read()
    stopwords = content.split(",")

emotion_features = Features({'text': Value('string')})
dataset = load_dataset("text", split="train", data_files="./data/wikidata/wikisent2.txt", features=emotion_features,   cache_dir='../baseline2/cache')
get_finetuning_data_wikipedia("../data/protoqa/triples/wikipedia.train.10.jsonl",
 "../data/protoqa/statement/train.statement.jsonl", "../data/protoqa/wikipedia/finetune_train_new_12.txt", "../data/protoqa/wikipedia/wiki_train_sentences.txt", debug=True, num_processes=48)



Using custom data configuration default-f63fa3181ad3428d
Reusing dataset text (../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)
 10%|█         | 1/10 [04:47<43:09, 287.72s/it]
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00000_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00001_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00002_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac6

Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00040_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00041_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00042_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00043_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00044_of_00200.arrow


Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00081_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00082_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00083_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00084_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00085_of_00200.arrow


Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00122_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00123_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00124_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00125_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00126_of_00200.arrow


Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00163_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00164_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00165_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00166_of_00200.arrow
Loading cached processed dataset at ../baseline2/cache/text/default-f63fa3181ad3428d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6cc7051ea8d980e9_00167_of_00200.arrow


...

#16:  57%|█████▊    | 23/40 [00:24<00:23,  1.36s/ba]











#12:  78%|███████▊  | 31/40 [00:25<00:09,  1.04s/ba]





#6:  55%|█████▌    | 22/40 [00:26<00:15,  1.14ba/s]












#13:  50%|█████     | 20/40 [00:25<00:41,  2.08s/ba]












#13:  52%|█████▎    | 21/40 [00:25<00:49,  2.60s/ba]

#2:  45%|████▌     | 18/40 [00:27<00:55,  2.50s/ba]

#2:  48%|████▊     | 19/40 [00:27<00:49,  2.34s/ba]



#4:  78%|███████▊  | 31/40 [00:26<00:04,  1.83ba/s]







#8:  57%|█████▊    | 23/40 [00:25<00:14,  1.16ba/s]

#2:  57%|█████▊    | 23/40 [00:27<00:15,  1.13ba/s]








#9:  75%|███████▌  | 30/40 [00:26<00:03,  2.78ba/s]








#9:  80%|████████  | 32/40 [00:26<00:04,  1.81ba/s]




#5:  78%|███████▊  | 31/40 [00:27<00:06,  1.32ba/s]


















 ... (more hidden) ...


















 ... (more hidden) ...






#7:  65%|██████▌   | 26/40 [00:27<00:26,  1.89s/ba]


#1:  72%|███████▎  | 29/40 [00:28<00:06,  1.63ba/s]














#15:  38%|███▊      | 15/40 [00:24<00:3

#14:  50%|█████     | 20/40 [00:32<00:42,  2.12s/ba]















#16:  78%|███████▊  | 31/40 [00:32<00:08,  1.05ba/s]















#16:  80%|████████  | 32/40 [00:32<00:09,  1.15s/ba]















#12: 100%|██████████| 40/40 [00:33<00:00,  1.18ba/s]




#4:  90%|█████████ | 36/40 [00:34<00:05,  1.25s/ba]







#8:  68%|██████▊   | 27/40 [00:33<00:22,  1.70s/ba]












#13:  88%|████████▊ | 35/40 [00:33<00:03,  1.60ba/s]




#5:  95%|█████████▌| 38/40 [00:34<00:01,  1.06ba/s]








#9:  95%|█████████▌| 38/40 [00:34<00:02,  1.10s/ba]


















 ... (more hidden) ...


#3:  52%|█████▎    | 21/40 [00:35<00:35,  1.87s/ba]






#7:  75%|███████▌  | 30/40 [00:35<00:19,  1.93s/ba]

#2:  92%|█████████▎| 37/40 [00:35<00:01,  2.43ba/s]

#2:  98%|█████████▊| 39/40 [00:35<00:00,  1.81ba/s]

















#18:  62%|██████▎   | 25/40 [00:34<00:33,  2.24s/ba]














#0:  75%|███████▌  | 30/40 [00:36<00:19,  1.91s/ba]













#14:  52%|█████▎    | 21/40 [00:34<00:3

#0:  90%|█████████ | 36/40 [00:45<00:05,  1.49s/ba]]





#6:  95%|█████████▌| 38/40 [00:45<00:02,  1.12s/ba]


















 ... (more hidden) ...






#50: 100%|██████████| 40/40 [00:42<00:00,  1.07s/ba]



#3:  95%|█████████▌| 38/40 [00:46<00:01,  1.58ba/s]

















#18:  80%|████████  | 32/40 [00:45<00:10,  1.34s/ba]

















#18:  82%|████████▎ | 33/40 [00:45<00:09,  1.38s/ba]













#14:  78%|███████▊  | 31/40 [00:45<00:10,  1.21s/ba]













#14:  80%|████████  | 32/40 [00:45<00:10,  1.28s/ba]














#15:  70%|███████   | 28/40 [00:43<00:15,  1.32s/ba]














#0:  92%|█████████▎| 37/40 [00:47<00:04,  1.56s/ba]]














#15:  75%|███████▌  | 30/40 [00:43<00:09,  1.03ba/s]





#6:  98%|█████████▊| 39/40 [00:46<00:01,  1.33s/ba]





#6: 100%|██████████| 40/40 [00:47<00:00,  1.18s/ba]



















 ... (more hidden) ...










#11:  75%|███████▌  | 30/40 [00:45<00:22,  2.23s/ba]






#7:  95%|█████████▌| 38/40 [00:47<00:

#198: 100%|██████████| 40/40 [00:58<00:00,  1.46s/ba]


KeyboardInterrupt: 